# タスク 2b: 抽象的なテキスト要約

このノートブックでは、大規模なドキュメント要約で発生する課題に対処します。入力テキストがモデルのコンテキスト長を超えたり、ハルシネーションを生成したり、メモリ不足エラーを引き起こしたりする可能性があります。

これらの問題を軽減するために、このノートブックでは、言語モデルを活用するアプリケーションを可能にするツールキットである [LangChain](https://python.langchain.com/docs/get_started/introduction.html) フレームワークを使用してプロンプトのチャンク化と連鎖化を使用するアーキテクチャを示します。

ユーザー ドキュメントがトークン制限を超えた場合のシナリオに対処するアプローチを探ります。チャンク化では、ドキュメントをコンテキスト長のしきい値以下のセグメントに分割してから、モデルに順番に入力します。これにより、プロンプトがチャンク間で連鎖され、以前のコンテキストが保持されます。このアプローチを適用して、通話のトランスクリプト、会議のトランスクリプト、書籍、記事、ブログ投稿、その他の関連コンテンツを要約します。


## Task 2b.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 2b.2: 長いテキストを要約する

### Boto3 を使用した LangChain の構成

このタスクでは、LangChain Bedrock クラスの LLM を指定する必要があり、推論用の引数を渡すことができます。

In [ ]:
# model configuration
from langchain_aws import ChatBedrock
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
chat = ChatBedrock(
    model_id=modelId,
    model_kwargs={
        "max_tokens": 2048,
        "temperature": 0,
        "top_p": 1
    },
    client=bedrock_client
)

## タスク 2b.3: 多数のトークンを含むテキスト ファイルの読み込み

このタスクでは、 [Amazon の CEO による 2022 年の株主への手紙の日本語版](https://www.aboutamazon.jp/news/company-news/ceo-andy-jassys-2022-letter-to-shareholders) のテキストファイルを読み込みます。

In [ ]:
#get tokens
shareholder_letter = "./2022-letter-jp.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()

# Anthropic のモデルでは下記は使用できない
#chat.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** 本来はここでテキストのトークンをカウントしたいところですが、Anthoropic のモデルの場合は Anthropic の API キーが必要なため、トークンのカウントはしません。

## タスク 2b.4: 長いテキストをチャンクに分割する

このタスクでは、テキストが長すぎてプロンプトに収まらないため、テキストを小さなチャンクに分割します。LangChain の `RecursiveCharacterTextSplitter` は、各チャンクのサイズが `chunk_size` より小さくなるまで、長いテキストを再帰的にチャンクに分割することをサポートします。テキストは `separators=["\n\n", "\n"]` でチャンクに分割され、各段落が複数のチャンクに分割されることを回避します。

チャンクごとに 4,000 文字を使用すると、各部分の要約を個別に取得できます。チャンク内のトークンまたは単語の断片の数は、テキストによって異なります。

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

print(
    f"Now we have {num_docs} documents"
)

## タスク 2b.5: チャンクを要約して結合する

このタスクでは、トークンの数が他のドキュメントと一貫していると仮定すると、問題なく実行できるはずです。LangChain の [load_summarize_chain](https://python.langchain.com/en/latest/use_cases/summarization.html) を使用してテキストを要約できます。`load_summarize_chain` は、`stuff`、`map_reduce`、および `refine` の 3 つの要約方法を提供します。

- `stuff`: すべてのチャンクを 1 つのプロンプトにまとめます。したがって、これはトークンの最大制限に達します。
- `map_reduce`: 各チャンクを要約し、要約を結合して、結合された要約を要約します。結合された要約が大きすぎる場合は、エラーが発生します。
- `refine`: 最初のチャンクを要約し、次に最初の要約を使用して 2 番目のチャンクを要約します。すべてのチャンクが要約されるまで、同じプロセスが繰り返されます。

map_reduce と refine はどちらも LLM を複数回呼び出し、最終的な要約を取得するのに時間がかかります。ここでは refine を試します。

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=chat, chain_type="refine", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** ドキュメントの数、Bedrock リクエストレートのクォータ、構成された再試行設定によっては、以下のチェーンの実行に時間がかかる場合があります。

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

これで、LangChain フレームワークでプロンプトのチャンク化と連鎖を使用して、長い入力テキストから生じる問題を軽減しながら、大きなドキュメントを要約する実験ができました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、**タスク 3** に進んでください。
